In [1]:
import numpy as np
import random

import math
import time

In [2]:
import torch as th

In [3]:
device = th.device('cuda')

In [4]:
from gene_mat import revers_mat, trans_rev, trans_mat

In [5]:
n = 200
tmp = np.array([range(1,n + 1)])
mat_list = []
for _ in range(20):
    p1,p2,p3,p4 = [random.randint(0,n) for _ in range(4)]
    mat_list.append(trans_rev(n, p1, p2, p3, p4))
tmp_tensor = th.tensor(tmp, dtype = th.float, device = device)
mat_tensor = th.tensor(np.array(mat_list), dtype = th.float, device = device)
test = th.matmul(tmp_tensor, mat_tensor)

In [6]:
p1,p2,p3,p4

(145, 53, 110, 48)

In [7]:
# n = 6
# count = 2**n*math.factorial(n)
# for _ in range(10):
#     num = []
#     genes_5 = []
#     g_5 = [list(range(1, n + 1))]
#     test = th.tensor(g_5, dtype = th.float, device = device)
#     num.append((1,-1))
#     genes_5.append(g_5)
    
#     mat_op_list = [trans_rev, trans_mat, revers_mat]
#     for i in range(1000):
#         op_list = [mat_op_list[random.randint(0,2)](
#     n, *[random.randint(0,n) for _ in range(4)]) 
#            for _ in range(200)]
        
#         mat_tensor = th.tensor(op_list, dtype = th.float, device = device)
#         test = th.matmul(test, mat_tensor)
        
#         g_res = test.cpu().numpy().astype(dtype=np.int32).tolist()
#         for g_5 in g_res:
#             if g_5 not in genes_5:
#                 genes_5.append(g_5)
#         l = len(genes_5)
#         if l > num[-1][0]:
#             num.append((l, i))
#             if l%100 == 0 or i % 1000 == 0: 
#                 print(time.ctime(), num[-1])
#         if l >= count:
#             break
#     print(time.ctime(), num[-1])

In [8]:
n = 10
print(time.ctime())
test = th.tensor(np.diag(np.repeat(1,n)), 
                 dtype = th.float, device = device)
mat_op_list = [trans_rev, trans_mat, revers_mat]

print(time.ctime())
for _ in range(10):    
    op_list = np.array([mat_op_list[random.randint(0,2)](
    n, *[random.randint(0,n) for _ in range(4)]) 
           for _ in range(20)])
    mat_tensor = th.tensor(op_list, dtype = th.float, device = device)
    test = th.matmul(test, mat_tensor)
#     print(test[0])
#     print(mat_tensor[0])
#     mat_tensor = mat_tensor[th.randperm(mat_tensor.shape[0])]
print(time.ctime())

Wed Nov  3 11:27:16 2021
Wed Nov  3 11:27:16 2021
Wed Nov  3 11:27:16 2021


In [9]:
th.matmul(test[0], test[0].T)

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], device='cuda:0')

In [10]:
test[0]

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], device='cuda:0')

In [11]:
test[0].T

tensor([[ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], device='cuda:0')

In [12]:
gene = th.tensor(np.arange(1, 11).reshape((1,10)), dtype = th.float, device = device)

In [13]:
gene

tensor([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]], device='cuda:0')

In [14]:
t1 = th.matmul(gene, test[0])

In [15]:
t1

tensor([[ 3.,  9., 10., -8., -5.,  7.,  4., -1.,  6.,  2.]], device='cuda:0')

In [16]:
t2 = th.matmul(t1, test[0].T)

In [17]:
t2

tensor([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]], device='cuda:0')

In [18]:
# p1,p2,p3,p4 = [random.randint(0,n) for _ in range(4)]
# for _ in range(2000):
#     op_type = random.randint(0,2)
#     if op_type == 0:
#         op_mat = trans_rev(n, p1, p2, p3, p4)
#     elif op_type == 1:
#         op_mat = trans_mat(n, p1, p2, p3, p4)
#     else:
#         op_mat = revers_mat(n, p1, p2)
#     op_list.append(op_mat)

---

In [19]:
from dcj_comp import dcj_dist

In [79]:
n, vol = 5, 2000
gene = th.tensor(np.arange(1, n+1).reshape((1,n)), dtype = th.float, device = device)
oriseq = gene.cpu().numpy().astype(np.int32).reshape(n,).tolist()

genome = []
genome.append(oriseq)

t_dist = [0 for _ in range(vol)]

In [80]:
test = th.tensor(np.diag(np.repeat(1,n)), 
                 dtype = th.float, device = device)
mat_op_list = [trans_rev, trans_mat, revers_mat]
rand_op = [2 for _ in range(vol)] #[random.randint(0,2) for _ in range(vol)]
param_op = [[random.randint(0,n) for _ in range(4)] for _ in range(vol)]
t_dist = np.add(t_dist, [1 if x == 2 else 2 for x in rand_op])

# op_list = np.array([mat_op_list[random.randint(0,2)](
# n, *[random.randint(0,n) for _ in range(4)]) 
#        for _ in range(20)])
op_list = np.array([mat_op_list[op](
n, *param) 
       for op, param in zip(rand_op, param_op)])


mat_tensor = th.tensor(op_list, dtype = th.float, device = device)
test = th.matmul(test, mat_tensor)
trans_seq = th.matmul(gene, test)

In [81]:
new_seq = trans_seq.cpu().numpy().astype(np.int32)
for x, d, o, op_param in zip(new_seq, t_dist, rand_op, param_op):
    if dcj_dist(oriseq, x[0])[-1] != d:
        print(x[0], '***\t', dcj_dist(oriseq, x[0])[-1], d, mat_op_list[o].__name__, op_param)
    if x[0].tolist() in genome:
        continue
    genome.append(x[0].tolist())

[-5 -4 -3 -2 -1] ***	 0 1 revers_mat [0, 5, 0, 0]
[-5 -4 -3 -2 -1] ***	 0 1 revers_mat [5, 0, 0, 4]
[1 2 3 4 5] ***	 0 1 revers_mat [5, 5, 1, 3]
[-5 -4 -3 -2 -1] ***	 0 1 revers_mat [5, 0, 0, 2]
[1 2 3 4 5] ***	 0 1 revers_mat [3, 3, 3, 2]
[1 2 3 4 5] ***	 0 1 revers_mat [4, 4, 1, 5]
[1 2 3 4 5] ***	 0 1 revers_mat [1, 1, 5, 1]
[1 2 3 4 5] ***	 0 1 revers_mat [4, 4, 0, 4]
[1 2 3 4 5] ***	 0 1 revers_mat [1, 1, 2, 2]
[1 2 3 4 5] ***	 0 1 revers_mat [1, 1, 1, 0]
[1 2 3 4 5] ***	 0 1 revers_mat [1, 1, 5, 1]
[1 2 3 4 5] ***	 0 1 revers_mat [1, 1, 2, 4]
[-5 -4 -3 -2 -1] ***	 0 1 revers_mat [5, 0, 4, 3]
[1 2 3 4 5] ***	 0 1 revers_mat [2, 2, 5, 3]
[1 2 3 4 5] ***	 0 1 revers_mat [2, 2, 4, 0]
[1 2 3 4 5] ***	 0 1 revers_mat [4, 4, 0, 3]
[1 2 3 4 5] ***	 0 1 revers_mat [2, 2, 4, 3]
[1 2 3 4 5] ***	 0 1 revers_mat [5, 5, 0, 1]
[-5 -4 -3 -2 -1] ***	 0 1 revers_mat [5, 0, 3, 2]
[1 2 3 4 5] ***	 0 1 revers_mat [3, 3, 4, 1]
[1 2 3 4 5] ***	 0 1 revers_mat [5, 5, 4, 5]
[1 2 3 4 5] ***	 0 1 revers_ma

In [39]:
rand_op

[1, 2, 0, 0, 2, 0, 0, 0, 2, 1, 0, 0, 2, 1, 2, 1, 2, 2, 2, 1]

In [61]:
trans_mat.__name__

'trans_mat'

In [25]:
dcj_dist([1,2,3,4,5], [-1, -3, -2,  4,  5])

(2, 2, 2)

In [33]:
new_seq

array([[[ 5,  3,  4,  1,  2]],

       [[ 1,  3,  4,  2,  5]],

       [[-3, -2, -1, -5, -4]],

       [[-4, -3,  1,  2,  5]],

       [[ 1,  3,  4,  2,  5]],

       [[-4, -5,  1,  2,  3]],

       [[ 1,  2,  3,  4,  5]],

       [[ 1,  2,  3,  4,  5]],

       [[ 1,  3,  4,  5, -2]],

       [[ 1,  2, -5, -4, -3]],

       [[ 1,  2,  3,  5,  4]],

       [[ 1, -3, -2,  4,  5]],

       [[-1,  2,  3,  4,  5]],

       [[ 1,  2, -4, -3,  5]],

       [[ 5, -3, -2, -1, -4]],

       [[-4,  1,  2,  3,  5]],

       [[ 1,  2,  3,  4,  5]],

       [[ 1, -5, -4, -3, -2]],

       [[ 3,  1,  2,  4,  5]],

       [[ 1,  2,  3,  4,  5]]], dtype=int32)

In [49]:
trans_rev(5, 0,3,0,3)

array([[ 0,  0, -1,  0,  0],
       [ 0, -1,  0,  0,  0],
       [-1,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  1]])

In [51]:
revers_mat(5, 3,3,2,4)

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]])

In [54]:
trans_mat(5, 2,2,4,4)

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]])

In [55]:
trans_rev(5, 3,3,0,3)

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]])

In [56]:
a=[1,2,3]
b=[4,5,6]
c=[6,7,8]
for x,y,z in zip(a,b,c):
    print(x,y,z)

1 4 6
2 5 7
3 6 8
